In [51]:
# библиотеки
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter

In [46]:
# ссылка и название программ по заданным фильтрам
programs_data = []

for page_n in range(1,26):
    # целевой сайт
    base_url = f'https://gisp.gov.ru/nmp/main/{page_n}?recommended=0&territory%5B%5D=6403441&territory%5B%5D=6403442&search_terms=&search_terms=&search_terms=&companyProblem%5B%5D=8788172&companyProblem%5B%5D=8788173&companyProblem%5B%5D=8788174&companyProblem%5B%5D=8788177&companyProblem%5B%5D=8788178&companyProblem%5B%5D=8788179&search_terms=&event=&search_terms=&search_terms=&search_terms=&search_terms=&search_terms=&measureActive=0&searchstr='
    r = requests.get(base_url)
    r.raise_for_status()  # Проверка HTTP ошибок
    soup = BeautifulSoup(r.text, 'html')

    # находим все ссылки на программы, которые расположены на сайте
    base = soup.find_all('a', target="_blank")

    for item in base:
        program_data = {
            'title': item.get('title'),  # Название из атрибута title
            'link': 'https://gisp.gov.ru' + item.get('href'),   # Ссылка
            }
        programs_data.append(program_data)


### Для каждой программы переходим по ссылке и извлекаем дополнительные данные (тип меры, поддерживаемые отрасли, назначение программы)

In [ ]:
for program in programs_data:
        full_url = program['link']
        
        # Загружаем страницу программы
        response = requests.get(full_url)
        response.raise_for_status()  # Проверка на ошибки HTTP
        
        soup = BeautifulSoup(response.text, 'html')
        
        # ИНФОРМАЦИЯ О ТИПЕ МЕРЫ
        # Ищем блок с таблицей характеристик
        info_general = soup.find('div', class_='info__general')
        
        if info_general:
            table = info_general.find('table')
            
            if table:
                # Извлекаем все строки таблицы
                rows = table.find_all('tr')
                
                for row in rows:
                    header = row.find('th')
                    if header:
                        header_text = header.get_text(strip=True)
                        
                        # 1. Тип меры
                        if 'Тип меры' in header_text:
                            type_cell = row.find('td')
                            if type_cell:
                                program['measure_type'] = type_cell.get_text(strip=True)
                        
                        # 2. Поддерживаемые отрасли
                        elif 'Поддерживаемые отрасли' in header_text:
                            industries_cell = row.find('td')
                            if industries_cell:
                                # Извлекаем все элементы списка отраслей
                                industries_list = industries_cell.find('ul', class_='hidden-list')
                                if industries_list:
                                    industries = []
                                    li_items = industries_list.find_all('li')
                                    for li in li_items:
                                        industry = li.get_text(strip=True)
                                        if industry:  # Проверяем, что текст не пустой
                                            industries.append(industry)
                                    program['supported_industries'] = industries
                                else:
                                    program['supported_industries'] = []
        else:
            # Если блок не найден, устанавливаем значения по умолчанию
            program['measure_type'] = 'Не указано'
            program['supported_industries'] = 'Не указано'
        
        # ДОПОЛНИТЕЛЬНО: можно извлечь и другую информацию
        # Например, назначение программы
        info_wrapper = soup.find('div', class_='info__wrapper')
        if info_wrapper:
            # Ищем заголовок "Назначение"
            for h2 in info_wrapper.find_all('h2'):
                if h2.get_text(strip=True) == 'Назначение':
                    # Берем следующий div с классом info__default
                    next_div = h2.find_next_sibling('div', class_='info__default')
                    if next_div:
                        program['purpose'] = next_div.get_text(strip=True, separator=' ')
                        break
        
        print(f"Обработано: {program['title']}")


Обработано: None
Обработано: Специальный инвестиционный контракт (СПИК 2.0)
Обработано: Программа ФРП 'Автокомпоненты'
Обработано: Программа ФРП 'Комплектующие изделия'
Обработано: Программа ФРП 'Маркировка товаров'
Обработано: Совместные займы ФРП с РФРП по программе 'Проекты лесной промышленности'
Обработано: Программа ФРП 'Проекты развития'
Обработано: Совместные займы ФРП с РФРП по программе 'Проекты развития'
Обработано: Совместные займы ФРП с РФРП по программе 'Комплектующие изделия'
Обработано: Программа ФРП 'Повышение производительности труда'
Обработано: Совместные займы ФРП с РФРП по программе 'Повышение производительности труда'
Обработано: None
Обработано: Программа ФРП 'Лизинг'
Обработано: Кластерная инвестиционная платформа (КИП)
Обработано: Механизм поддержки завершающихся проектов по производству приоритетной продукции
Обработано: Субсидии организациям авиационной промышленности, на компенсацию затрат на уплату купонного дохода по облигационным займам, привлеченным с пр

In [48]:
programs_data

[{'title': None,
  'link': 'https://gisp.gov.ru/nmp//main/?recommended=1',
  'measure_type': 'Не указан',
  'supported_industries': []},
 {'title': 'Специальный инвестиционный контракт (СПИК 2.0)',
  'link': 'https://gisp.gov.ru/nmp/measure/10511015',
  'measure_type': 'Регуляторная поддержка (налоговые, таможенные, инвестиционные льготы и т.д.)',
  'supported_industries': ['Авиационная промышленность',
   'Автомобильная промышленность',
   'Легкая промышленность',
   'Лесопромышленный комплекс',
   'Машиностроение для пищевой и перерабатывающей промышленности',
   'Медицинская промышленность',
   'Нефтегазовое машиностроение',
   'Промышленность боеприпасов и спецхимии',
   'Радиоэлектронная промышленность',
   'Сельскохозяйственное машиностроение',
   'Станкоинструментальная промышленность',
   'Судостроительная промышленность',
   'Тяжелое машиностроение',
   'Фармацевтическая промышленность',
   'Химическая промышленность',
   'Энергетическое машиностроение',
   'Народные художеств

# Теперь подсчитаем, сколько раз встречается та или иная отрасль

In [ ]:
all_industries_list = []
for industries in df['supported_industries'].dropna():
    if isinstance(industries, list):
        all_industries_list.extend(industries)

industry_counts = Counter(all_industries_list)

# Сортируем по убыванию частоты
sorted_counts = industry_counts.most_common()

print("Частота упоминания отраслей:")
for industry, count in sorted_counts:
    print(f"{industry}: {count} раз")

Частота упоминания отраслей:
Автомобильная промышленность: 106 раз
Сельскохозяйственное машиностроение: 87 раз
Легкая промышленность: 84 раз
Авиационная промышленность: 83 раз
Станкоинструментальная промышленность: 83 раз
Медицинская промышленность: 82 раз
Радиоэлектронная промышленность: 81 раз
Химическая промышленность: 80 раз
Энергетическое машиностроение: 79 раз
Фармацевтическая промышленность: 78 раз
Лесопромышленный комплекс: 77 раз
Судостроительная промышленность: 77 раз
Тяжелое машиностроение: 76 раз
Нефтегазовое машиностроение: 74 раз
Машиностроение для пищевой и перерабатывающей промышленности: 72 раз
Электротехническая промышленность: 71 раз
Железнодорожная промышленность: 69 раз
Цветная металлургия, промышленность редких и редкоземельных металлов: 66 раз
Черная металлургия, трубная промышленность и металлоконструкции: 64 раз
Промышленность строительных и нерудных материалов: 61 раз
Кабельная промышленность: 59 раз
Народные художественные промыслы: 58 раз
Промышленность пере

# Преобразуем список программ в DataFrame и экспортируем полученные результаты в формате csv. 
Также отметим индексом 1/0, встречается ли та или иная отрасль в программе или нет

In [ ]:

df = pd.DataFrame(programs_data)

# Получаем ВСЕ уникальные отрасли из ВСЕХ программ
all_industries = set()

for industries_list in df['supported_industries']:
    if isinstance(industries_list, list) and industries_list:  # Проверяем, что это непустой список
        all_industries.update(industries_list)

# Преобразуем в сортированный список
all_industries = sorted(list(all_industries))
print(f"Найдено {len(all_industries)} уникальных отраслей")

# Создаём новые столбцы для каждой отрасли
for industry in all_industries:
    df[industry] = df['supported_industries'].apply(
        lambda x: 1 if isinstance(x, list) and industry in x else 0
    )

Найдено 56 уникальных отраслей


In [ ]:
name = input('Введи имя файла для сохранения')
df.to_csv(name, index = False, encoding='utf-8')
print("Данные экспортированы в", name)

Данные экспортированы в  GISP_GOV_STATS
